## Remap events.

This notebook demonstrates how to use HED tools to remap event files. The idea of
remapping is that each unique combination of values in a set of **key** columns is
mapped into a specified combination of values in a set of **target** columns.

Table 1 shows excerpt from a sample events file from the Sternberg dataset. We wish
to remap the codes in the `type` column to a combination of values in new columns
called `event_type`, `task_role`, and `letter` columns.

Since the event files don't contain these columns, the mapping must insert these columns
as part of the mapping. Note the `latency` gives the sample number of the event, rather
than the offset in seconds as required by BIDS. The required `offset` and `duration`
columns would need to be inserted to make this events file BIDS-compliant.

**Table 1:** Excerpt from a Sternberg working memory event file.

| pres_trial | stimulus | type  | latency |
| ---------- | -------- | ----- | ------- |
| 1          | Picture  | nonWM | 1430    |
| 1          | Picture  | gD    | 2741    |
| 1          | Picture  | gM    | 3102    |
| 1          | Picture  | gB    | 3462    |
| 1          | Picture  | Y     | 3822    |
| 1          | Picture  | gH    | 4183    |

Table 2 shows an excerpt of the mapping template which has already been created.

**Table 2:** Excerpt of a map designed to map the `type` column into
a combination of columns `event_type`, `task_role`, and `letter`.

| type	  | event_type  | task_role        | letter |
| ------- | ----------- | ---------------- | ------ |
| A	      | show_letter | target           | A      |
| gD      | show_letter | non-target       | D      |
| Y       | show_letter | target           | E      |
| rK      | show_letter | probe            | K      |
| nonWM   | show_cross  | fixate           | +      |
| correct | sound_beep  | correct_feedback | n/a    |
| 1       | right_click | in_group         | n/a    |

#### Set dataset location

The example used in this notebook is a Sternberg memory dataset. Some data files and
the previously created template are located in the the `data/sternberg` directory.

#### Set the data directory

In [1]:
data_directory = "../data/sternberg_data"

#### Create a `KeyMap` to do the mapping

In [2]:
import os
from hed.tools import KeyMap
key_cols = ['type']
target_cols = ['event_type', 'task_role', 'letter']
key_map = KeyMap(key_cols, target_cols, name='sternberg_remap')


#### Read the pre-created mapping template

If you don't have a mapping for your data, you should use the
`create_template.ipynb` notebook first.

The following reads the template mapping unique values of `type` to
combinations of `event_type`, `task_role`, and `letter`.


In [3]:
template_file = os.path.join(data_directory, "sternberg_map.tsv")
print(f"Template file is {template_file}")
duplicates = key_map.update(template_file)
if duplicates:
    print(f"Warning: this mapping has {len(duplicates)} duplicate key entries")


Template file is ../data/sternberg\sternberg_map.tsv


#### Get a list of event files to be remapped

We will remap the event files in the `data_directory` whose names start with
`sternberg` and end with `_events.tsv`.


In [4]:
from hed.tools import get_file_list
event_file_list = get_file_list(data_directory, types=[".tsv"], suffix="_events")


#### Remap event files

Remapping consists of replacing/adding columns (in this case adding columns).

In [5]:
from hed.tools import generate_filename
for file in event_file_list:
    print(f"Remapping {file}...")
    df_new, missing = key_map.remap(file)
    if missing:
        print(f"The keys {str(missing)} were not in the map for {file}")
    file_name = generate_filename(file, prefix="remapped", extension='.tsv')
    file_path = os.path.join(data_directory, file_name)
    print(f"...Writing remapped file to {file_path}")
    df_new.to_csv(file_path, sep='\t', index=False)

Remapping ../data/sternberg\remapped_data_sternberg_remapped_data_sternberg_sternberg_no_quotes_events.tsv...
...Writing remapped file to ../data/sternberg\remapped_data_sternberg_remapped_data_sternberg_remapped_data_sternberg_sternberg_no_quotes_events.tsv
Remapping ../data/sternberg\remapped_data_sternberg_remapped_data_sternberg_sternberg_test_events.tsv...
...Writing remapped file to ../data/sternberg\remapped_data_sternberg_remapped_data_sternberg_remapped_data_sternberg_sternberg_test_events.tsv
Remapping ../data/sternberg\remapped_data_sternberg_remapped_data_sternberg_sternberg_with_quotes_events.tsv...
...Writing remapped file to ../data/sternberg\remapped_data_sternberg_remapped_data_sternberg_remapped_data_sternberg_sternberg_with_quotes_events.tsv
Remapping ../data/sternberg\sternberg_no_quotes_events.tsv...
...Writing remapped file to ../data/sternberg\remapped_data_sternberg_sternberg_no_quotes_events.tsv
Remapping ../data/sternberg\sternberg_test_events.tsv...
...Writin